# LPM Window 별로 분석
## 이렇게 하는 이유
* 각 단어의 LPM을 절대적으로 구할 수 있다면 가장 좋겠지만, 단어 발화 시에 앞 뒤로 생기는 휴지가 있기 때문에 현실적으로 

In [5]:
import json

stt_file_path = './samples/kss_concatenated_script_sample.json'
with open(stt_file_path, 'r') as f:
    stt_script = json.load(f)

segments = stt_script['segments']
print('Number of segments:', len(segments))

Number of segments: 93


In [25]:
from collections import Counter


WINDOW_SIZE = 10


def get_lpm(start_time: int, end_time: int, letter_count: int):
    return letter_count / (end_time - start_time) * 1000 * 60


word_index = 0
for s_idx, segment in enumerate(segments):
    for w_idx in range(len(segment["words"])):
        segments[s_idx]["words"][w_idx] = (
            *segments[s_idx]["words"][w_idx],
            word_index,
        )
        word_index += 1

words = []
# (빠르게 잡힌 횟수, 느리게 잡힌 횟수)
word_speed_count = [(0, 0) for _ in range(word_index - 1)]


for segment in segments:
    words.extend(segment["words"])

for idx in range(len(words) - WINDOW_SIZE):
    start_time = words[idx][0]
    end_time = words[idx + WINDOW_SIZE - 1][1]
    letter_count = sum([len(w[2]) for w in words[idx : idx + WINDOW_SIZE]])

    lpm = get_lpm(start_time, end_time, letter_count)
    if lpm > 400:
        # print("TOOFASt",lpm)
        word_speed_count[idx : idx + WINDOW_SIZE] = [
            (wsc[0] + 1, wsc[1]) for wsc in word_speed_count[idx : idx + WINDOW_SIZE]
        ]
    elif lpm < 300:
        # print("tooslow",idx,lpm)
        word_speed_count[idx : idx + WINDOW_SIZE] = [
            (wsc[0], wsc[1] + 1) for wsc in word_speed_count[idx : idx + WINDOW_SIZE]
        ]

final_list = [(x[0], y[2], x[1], y[0] / 1000) for x, y in zip(word_speed_count, words)]
# final_list.sort(key=lambda x: x[0], reverse=True)
print(
    Counter([x[0] for x in final_list]).most_common(10),
    Counter([x[2] for x in final_list]).most_common(10),
)
print("\n".join(map(str, final_list)))

[(0, 700), (1, 155), (2, 58), (4, 33), (3, 27), (6, 7), (5, 4), (7, 2)] [(0, 698), (1, 95), (2, 93), (4, 34), (3, 28), (6, 11), (7, 11), (5, 6), (9, 6), (8, 4)]
(1, '독일', 0, 0.47)
(1, '제국의', 0, 0.77)
(2, '철혈재상', 0, 1.15)
(2, '비스마르크가', 0, 1.69)
(2, '독일을', 0, 2.55)
(2, '통일하고', 0, 2.95)
(2, '보니', 0, 3.4)
(2, '세계의', 0, 4.01)
(2, '식민지는', 0, 4.4)
(2, '이미', 0, 4.97)
(1, '열강들의', 0, 5.39)
(1, '차지였습니다.', 0, 6.11)
(0, '단독', 0, 7.55)
(0, '진출은', 0, 7.93)
(0, '무리라고', 0, 8.3)
(0, '판단한', 0, 8.74)
(0, '비스마르크는', 0, 9.13)
(0, '동양에서', 0, 9.93)
(0, '협력자를', 0, 10.63)
(0, '찾았습니다.', 0, 11.33)
(0, '당시', 0, 12.77)
(0, '중국의', 0, 13.15)
(0, '실력자인', 0, 13.56)
(0, '이용장이', 0, 14.17)
(0, '적합해', 0, 15.13)
(0, '보였습니다.', 0, 15.95)
(0, '비스마르크는', 0, 17.19)
(0, '그의', 0, 17.85)
(0, '환심을', 0, 18.1)
(0, '사기', 1, 18.52)
(0, '위해', 1, 18.81)
(0, '독일산', 2, 19.45)
(0, '순종', 2, 20.09)
(0, '셰퍼드', 2, 20.57)
(0, '두', 2, 21.01)
(0, '마리를', 2, 21.17)
(0, '고르고', 2, 22.01)
(0, '골라', 2, 22.41)
(0, '선물로', 2, 22.99)
(0, '보냈습니다.', 1, 23.63)
(0